In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('data/2020.csv',encoding='euc-kr')
df.drop(columns=['등록일자'],inplace=True)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200101,1호선,종각,20427,16301
1,20200101,1호선,시청,12126,10516
2,20200101,우이신설선,신설동,892,828
3,20200101,우이신설선,보문,917,855
4,20200101,우이신설선,성신여대입구(돈암),2010,2363


In [12]:
cols = list(df.columns)[:3]
target = list(df.columns)[3:]
df_list = []
# 반복문에 해당되지 않는 범위는 수동으로 구해서 
for i in range(6, 28, 7): # 사용할때 여기 수정 해당 월에 맞게   여긴 1월2일이 월요일이라 2일 부터 시작 30,31일 데이터는 추가로 더해야함
    start_date = 20200100 + i   # 여기도 조건에 맞게 수정
    end_date = start_date + 5
    df_temp = df[(df['사용일자'] >= start_date) & (df['사용일자'] < end_date)].copy()
    df_temp = df_temp.groupby(cols)[target].agg('sum').reset_index()
    df_list.append(df_temp)
df_1 = pd.concat(df_list, axis=0)
df_1['사용일자'] = 202001

df_1 = df_1.groupby(cols)[target].agg('sum').reset_index()

In [13]:
df_1.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,202001,1호선,동대문,265659,281449
1,202001,1호선,동묘앞,175273,183094
2,202001,1호선,서울역,1142828,1073944
3,202001,1호선,시청,530566,539048
4,202001,1호선,신설동,319808,311230


In [14]:
df_1.shape

(595, 5)

In [15]:
# 이렇게 12개월치 구한거
# df_res_list = [df_1,df_2,df_3 ....df_12] 리스트 안에 넣음
# df_res = pd.concat(df_res_list, axis=0) 합침
# 이거에 이제 역명 '(' 지우고 하는 작업하고
# 호선별로 나눈다음에
# line_merge 코드 짜둔거 사용해서 합치면 됨

In [16]:
# 수동
df1 = df[(df.사용일자 >= 20200101)&(df.사용일자 < 20200104)].copy()
df1.groupby(cols)[target].agg('sum').reset_index()
df1.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200101,1호선,종각,20427,16301
1,20200101,1호선,시청,12126,10516
2,20200101,우이신설선,신설동,892,828
3,20200101,우이신설선,보문,917,855
4,20200101,우이신설선,성신여대입구(돈암),2010,2363


In [18]:
df_1 = pd.concat([df_1, df1], axis=0)
df_1.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,202001,1호선,동대문,265659,281449
1,202001,1호선,동묘앞,175273,183094
2,202001,1호선,서울역,1142828,1073944
3,202001,1호선,시청,530566,539048
4,202001,1호선,신설동,319808,311230


In [21]:
df_1.to_csv('data/aaaaa.csv', index=False)

In [ ]:
# 1~12월 다 합치고 불러와서 아래 실행
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    # frame = df[df['호선명']==line].copy()
    frame['총 승차인원'] = frame.loc[:,['승차총승객수']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['하차총승객수']].sum(axis=1)

    frame.to_csv(f'data/main/{line}.csv',index=False,encoding='utf-8')

In [ ]:
# 저장 위치 고치기
line_info = [
    (['./1호선.csv', './경부선.csv', './경원선.csv', './경인선.csv', './장항선.csv'], '1호선'),
    (['./2호선.csv'], '2호선'),
    (['./3호선.csv', './일산선.csv'], '3호선'),
    (['./4호선.csv', './과천선.csv', './안산선.csv'], '4호선'),
    (['./5호선.csv'], '5호선'),
    (['./6호선.csv'], '6호선'),
    (['./7호선.csv'], '7호선'),
    (['./8호선.csv'], '8호선'),
    (['./9호선.csv', './9호선2단계.csv', './9호선2~3단계.csv'], '9호선'),
    (['./수인선.csv', './분당선.csv'], '수인분당선'),
    (['./경의선.csv', './용산선.csv''./경원선.csv', './중앙선.csv'], '경의중앙선')

]

for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    res.to_csv(f'./lines/{line_name}.csv', index=False)
